<img src="https://pmcvariety.files.wordpress.com/2018/04/pubg.jpg?w=1000&h=563&crop=1" alt="PUBG" style="width: 750px;"/>

### 이 커널은 다음 배틀그라운드 데이터에서 다음 피쳐를 시각화하여 인사이트를 얻고자 한다.
- [The Killers](#The-Killers)
- [The Runners](#The-Runners)
- [The Drivers](#The-Drivers)
- [The Swimmers](#The-Swimmers)
- [The Healers](#The-Healers)
- [Solos, Duos and Squads](#Solos,-Duos-and-Squads)
- [Correlation](#Pearson-correlation-between-variables)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 데이터 불러오기
train = pd.read_csv('../input/pubg-finish-placement-prediction/train_V2.csv')

In [ ]:
train.info()

## Feature에 대한 설명
- **groupId** - 한 매치 안에서 서로 다른 유저를 구분하기 위한 ID. 같은 유저 그룹일지라도 다른 매치에서는 매번 다른 ID가 주어진다.
- **matchId** - 매치 ID. 트레인과 테스트 간에 중복되는 ID는 존재하지 않는다.
- **assists** - 팀원이 죽인 적들 중에 데미지를 가해 도운 횟수 
- **boosts** - 사용된 부스트 아이템의 개수
- **damageDealt** - 가한 데미지의 총합. 스스로 가한 데미지는 제외.
- **DBNOs** - 기절시킨 적의 수
- **headshotKills** - 헤드샷으로 죽은 적의 수
- **heals** - 사용된 치료 아이템의 개수
- **killPlace** - 매치에서 킬 수 기준으로 집계한 등수
- **killPoints** - 킬 랭킹 기반의 포인트
- **kills** - 죽인 적의 횟수
- **killStreaks** - 짧은 시간 내에 가장 많이 적을 죽인 횟수
- **maxPlace** - 매치 내에서 가장 낮은 등수. 가끔 데이터가 등수를 거르기도해서 때문에 총 그룹의 수와 일치하지 않을 수 있음
- **numGroups** - 매치 내에서 총 그룹의 수
- **revives** - 유저가 팀원을 살린 횟수
- **rideDistance** - 차량을 통해 이동한 총거리 (미터)
- **roadKills** - 차량 탑승 도중에 적을 죽인 횟수
- **swimDistance** - 유저가 수영한 총거리 (미터)
- **teamKills** - 유저가 같은 팀원을 죽인 횟수
- **vehicleDestroys** - 이동차량을 파괴한 횟수
- **walkDistance** - 걸어서 이동한 총거리 (미터)
- **weaponsAcquired** - 총 획득한 무기의 수
- **winPoints** - 승리에 기반한 유저의 랭킹 포인트
- **winPlacePerc** - 백분위수에 기반한 타겟의 예측. 1은 1등을 의미하며 0은 마지막 등수를 의미한다. 

In [ ]:
# 상위 5개 row들
train.head()

## The Killers

<img src="https://i.ytimg.com/vi/rnAeX795Jn0/maxresdefault.jpg" alt="The Killers" style="width: 700px;"/>

In [ ]:
print("유저는 평균적으로 {:.4f}명의 킬을 하며, 99%의 유저들은 {} 이하의 킬을 한다. 반면 최대 많이 킬을 한 횟수는 {}이다.".format(train['kills'].mean(),train['kills'].quantile(0.99), train['kills'].max()))

In [ ]:
# 킬 카운트를 시각화 해보자.
data = train.copy()
data.loc[data['kills'] > data['kills'].quantile(0.99)] = '8+'
plt.figure(figsize=(15,10))
sns.countplot(data['kills'].astype('str').sort_values())
plt.title("Kill Count",fontsize=15)
plt.show()

- 대부분의 사람들은 1킬도 하지 못한다는걸 알 수 있다. 그렇다면 최소한 데미지라도 가하는지 확인해보자.

In [ ]:
data = train.copy()
data = data[data['kills']==0]
plt.figure(figsize=(15,10))
plt.title("Damage Dealt by 0 killers",fontsize=15)
sns.distplot(data['damageDealt'])
plt.show()

- 데미지조차 가하지 못하는 사람이 대부분이라는걸 알 수 있다.

In [ ]:
print("{}의 유저({:.4f}%)는 킬을 하지도 않고도 우승했다".format(len(data[data['winPlacePerc']==1]), 100*len(data[data['winPlacePerc']==1])/len(train)))

data1 = train[train['damageDealt'] == 0].copy()
print("{}의 유저({:.4f}%)는 데미지를 가하지 않고 우승했다".format(len(data1[data1['winPlacePerc']==1]), 100*len(data1[data1['winPlacePerc']==1])/len(train)))

- 우승한 횟수와 킬 수 간의 관계를 시각화를 해보자.

In [ ]:
sns.jointplot(x="winPlacePerc", y="kills", data=train, height=10, ratio=3, color="r")
plt.show()

- 명백하게도 킬 수 와 우승은 양의 상관관계가 존재한다. 그렇다면 킬 수를 기반으로 그룹화 하여 살펴보자.  

In [ ]:
kills = train.copy()

kills['killsCategories'] = pd.cut(kills['kills'], [-1, 0, 2, 5, 10, 60], labels=['0_kills','1-2_kills', '3-5_kills', '6-10_kills', '10+_kills'])

plt.figure(figsize=(15,8))
sns.boxplot(x="killsCategories", y="winPlacePerc", data=kills)
plt.show()

## The Runners

<img src="https://steemitimages.com/DQmRmYLRxu1vUhVtnFAA6bHFbShtr7Wdv1wLrPjdxbRZsjc/maxresdefault%20(2).jpg" alt="The Runners" style="width: 700px;"/>

In [ ]:
print("평균적으로 유저는 {:.1f}m를 뛰며, 99%의 유저는 {}m 이하를 뛰었다. 반면 마라톤 챔피언은 {}m를 뛰었다.".format(train['walkDistance'].mean(), train['walkDistance'].quantile(0.99), train['walkDistance'].max()))

In [ ]:
data = train.copy()
data = data[data['walkDistance'] < train['walkDistance'].quantile(0.99)]
plt.figure(figsize=(15,10))
plt.title("Walking Distance Distribution",fontsize=15)
sns.distplot(data['walkDistance'])
plt.show()

In [ ]:
print("{}의 유저({:.4f}%)는 0미터를 걸었다. 이것이 의미하는 것은 유저가 발을 딛기 전에도 죽었다는 것이다.".format(len(data[data['walkDistance'] == 0]), 100*len(data1[data1['walkDistance']==0])/len(train)))

In [ ]:
sns.jointplot(x="winPlacePerc", y="walkDistance",  data=train, height=10, ratio=3, color="lime")
plt.show()

- 명백하게 뛴 거리와 등수는 상관관계가 존재한다.

## The Drivers

<img src="http://cdn.gamer-network.net/2018/metabomb/pubghowtodrivecarsandbikes.jpg" alt="The Drivers" style="width: 700px;"/>

In [ ]:
print("평균적으로 유저는 {:.1f}m를 운전하며, 99%의 유저는 {}m 이하를 운전했다. 반면 최장거리 운전자는 {}m를 운전했다.".format(train['rideDistance'].mean(), train['rideDistance'].quantile(0.99), train['rideDistance'].max()))

In [ ]:
data = train.copy()
data = data[data['rideDistance'] < train['rideDistance'].quantile(0.9)]
plt.figure(figsize=(15,10))
plt.title("Ride Distance Distribution",fontsize=15)
sns.distplot(data['rideDistance'])
plt.show()

In [ ]:
print("{}의 유저({:.4f}%)는 0미터를 운전했다. 이 뜻은 그들이 아직 운전면허가 없다는 말이다.".format(len(data[data['rideDistance'] == 0]), 100*len(data1[data1['rideDistance']==0])/len(train)))

In [ ]:
sns.jointplot(x="winPlacePerc", y="rideDistance", data=train, height=10, ratio=3, color="y")
plt.show()

- 운전거리와 등수는 작은 양의 상관관계가 존재한다.

- 경험상 차량을 파괴하는 사람은 잘하는 유저일 확률이 높다. 확인해보자.

In [ ]:
f,ax1 = plt.subplots(figsize =(20,10))
sns.pointplot(x='vehicleDestroys',y='winPlacePerc',data=data,color='#606060',alpha=0.8)
plt.xlabel('Number of Vehicle Destroys',fontsize = 15,color='blue')
plt.ylabel('Win Percentage',fontsize = 15,color='blue')
plt.title('Vehicle Destroys/ Win Ratio',fontsize = 20,color='blue')
plt.grid()
plt.show()

- 시각화가 증명하듯, 차량을 파괴하는 유저는 높은 등수로 마무리할 확률이 높다

## The Swimmers

<img src="https://i.ytimg.com/vi/tQxzsE0DijQ/maxresdefault.jpg" alt="The Swimmers" style="width: 700px;"/>

In [ ]:
print("유저는 평균적으로 {:.1f}m를 수영하며, 99%의 유저는 {}m이하를 수영했다. 반면 수영 챔피언은 {}m를 수영했다.".format(train['swimDistance'].mean(), train['swimDistance'].quantile(0.99), train['swimDistance'].max()))

In [ ]:
data = train.copy()
data = data[data['swimDistance'] < train['swimDistance'].quantile(0.95)]
plt.figure(figsize=(15,10))
plt.title("Swim Distance Distribution",fontsize=15)
sns.distplot(data['swimDistance'])
plt.show()

- 대부분의 유저는 수영을 하지 않는다. 수영 거리로 그룹화하여 살펴보자

In [ ]:
swim = train.copy()

swim['swimDistance'] = pd.cut(swim['swimDistance'], [-1, 0, 5, 20, 5286], labels=['0m','1-5m', '6-20m', '20m+'])

plt.figure(figsize=(15,8))
sns.boxplot(x="swimDistance", y="winPlacePerc", data=swim)
plt.show()

- 수영을 하면 할수록 높은 등수로 마무리할 확률이 높다는걸 확인할 수 있다. 

## The Healers

<img src="https://i.ytimg.com/vi/xfI9XljX51k/maxresdefault.jpg" alt="The Healers" style="width: 700px;"/>

In [ ]:
print("평균적으로 유저는 {:.1f}개의 치료 아이템을 사용하며, 99%의 유저는 {}개 이하를 사용한다. 반면 가장 많이 치료 아이템을 사용한 유저는 {}개를 사용했다.".format(train['heals'].mean(), train['heals'].quantile(0.99), train['heals'].max()))
print("평균적으로 유저는 {:.1f}개의 부스트 아이템을 사용하며, 99%의 유저는 {}개 이하를 사용한다. 반면 가장 많이 부스트 아이템을 사용한 유저는 {}개를 사용했다.".format(train['boosts'].mean(), train['boosts'].quantile(0.99), train['boosts'].max()))

In [ ]:
data = train.copy()
data = data[data['heals'] < data['heals'].quantile(0.99)]
data = data[data['boosts'] < data['boosts'].quantile(0.99)]

f,ax1 = plt.subplots(figsize =(20,10))
sns.pointplot(x='heals',y='winPlacePerc',data=data,color='lime',alpha=0.8)
sns.pointplot(x='boosts',y='winPlacePerc',data=data,color='blue',alpha=0.8)
plt.text(4,0.6,'Heals',color='lime',fontsize = 17,style = 'italic')
plt.text(4,0.55,'Boosts',color='blue',fontsize = 17,style = 'italic')
plt.xlabel('Number of heal/boost items',fontsize = 15,color='blue')
plt.ylabel('Win Percentage',fontsize = 15,color='blue')
plt.title('Heals vs Boosts',fontsize = 20,color='blue')
plt.grid()
plt.show()

In [ ]:
sns.jointplot(x="winPlacePerc", y="heals", data=train, height=10, ratio=3, color="lime")
plt.show()

In [ ]:
sns.jointplot(x="winPlacePerc", y="boosts", data=train, height=10, ratio=3, color="blue")
plt.show()

- 치료 아이템과 부스트 아이템은 등수와 연관성이 깊다. 그 중에도 부스트 아이템이 더 연관성이 있다.

## Solos, Duos and Squads

- 게임 모드는 3개가 있다. 유저는 혼자하거나, 친구와 듀오를 이루거나 유저를 제외한 3명과 팀을 이룰 수도 있다.
- 따라서 듀오 모드일 경우 한 매치 내에서 팀의 총 수는 최대 50이며, 스쿼드 모드일 경우 최대 25이다.

In [ ]:
solos = train[train['numGroups']>50]
duos = train[(train['numGroups']>25) & (train['numGroups']<=50)]
squads = train[train['numGroups']<=25]
print("{} ({:.2f}%) 솔로 게임이 존재하며, {} ({:.2f}%) 듀오 게임이, 그리고 {} ({:.2f}%) 스쿼드 게임이 존재한다.".format(len(solos), 100*len(solos)/len(train), len(duos), 100*len(duos)/len(train), len(squads), 100*len(squads)/len(train),))

In [ ]:
f,ax1 = plt.subplots(figsize =(20,10))
sns.pointplot(x='kills',y='winPlacePerc',data=solos,color='black',alpha=0.8)
sns.pointplot(x='kills',y='winPlacePerc',data=duos,color='#CC0000',alpha=0.8)
sns.pointplot(x='kills',y='winPlacePerc',data=squads,color='#3399FF',alpha=0.8)
plt.text(37,0.6,'Solos',color='black',fontsize = 17,style = 'italic')
plt.text(37,0.55,'Duos',color='#CC0000',fontsize = 17,style = 'italic')
plt.text(37,0.5,'Squads',color='#3399FF',fontsize = 17,style = 'italic')
plt.xlabel('Number of kills',fontsize = 15,color='blue')
plt.ylabel('Win Percentage',fontsize = 15,color='blue')
plt.title('Solo vs Duo vs Squad Kills',fontsize = 20,color='blue')
plt.grid()
plt.show()

- 흥미로운 점은 솔로와 듀오 게임은 비슷한 양상을 보이지만, 스쿼드 게임은 킬 수가 그렇게 중요해 보이지 않는다.

In [ ]:
f,ax1 = plt.subplots(figsize =(20,10))
sns.pointplot(x='DBNOs',y='winPlacePerc',data=duos,color='#CC0000',alpha=0.8)
sns.pointplot(x='DBNOs',y='winPlacePerc',data=squads,color='#3399FF',alpha=0.8)
sns.pointplot(x='assists',y='winPlacePerc',data=duos,color='#FF6666',alpha=0.8)
sns.pointplot(x='assists',y='winPlacePerc',data=squads,color='#CCE5FF',alpha=0.8)
sns.pointplot(x='revives',y='winPlacePerc',data=duos,color='#660000',alpha=0.8)
sns.pointplot(x='revives',y='winPlacePerc',data=squads,color='#000066',alpha=0.8)
plt.text(14,0.5,'Duos - Assists',color='#FF6666',fontsize = 17,style = 'italic')
plt.text(14,0.45,'Duos - DBNOs',color='#CC0000',fontsize = 17,style = 'italic')
plt.text(14,0.4,'Duos - Revives',color='#660000',fontsize = 17,style = 'italic')
plt.text(14,0.35,'Squads - Assists',color='#CCE5FF',fontsize = 17,style = 'italic')
plt.text(14,0.3,'Squads - DBNOs',color='#3399FF',fontsize = 17,style = 'italic')
plt.text(14,0.25,'Squads - Revives',color='#000066',fontsize = 17,style = 'italic')
plt.xlabel('Number of DBNOs/Assits/Revives',fontsize = 15,color='blue')
plt.ylabel('Win Percentage',fontsize = 15,color='blue')
plt.title('Duo vs Squad DBNOs, Assists, and Revives',fontsize = 20,color='blue')
plt.grid()
plt.show()

- DBNO는 기절 횟수를 의미한다. 기절은 팀원이 살릴 수 있는(revive) 오직 듀오나 스쿼드에만 발생한다. 
- 따라서 기절당한 유저는 죽거나(die) 같은 팀원에 의해 살아날 수 있다(revive). 
- assist 또한 듀오나 스쿼드에서만 발생한다. 적을 죽일 때 얼마나 관여를 했는지 보여주는 지표이다.

## Pearson correlation between variables

In [ ]:
f,ax = plt.subplots(figsize=(15, 15))
sns.heatmap(train.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)
plt.show()

- 타겟 변수 (winPlacePerc)와의 상관계수를 확인해보면, 몇몇의 변수는 중간에서 높은 상관성을 보인다.
- 가장 높은 양의 상관계수는 walkDistance이고, 가장 높은 음의 상관계수는 killPlace이다.

**가장 높은 상관성을 보이는 5개의 변수를 좀 더 깊게 살펴보자.**

In [ ]:
k = 5 # 히트맵을 위한 변수의 개수
f,ax = plt.subplots(figsize=(11, 11))
cols = train.corr().nlargest(k, 'winPlacePerc')['winPlacePerc'].index
cm = np.corrcoef(train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

- 위의 5개 변수 killPlace 변수간의 관계를 시각화해보자

In [ ]:
sns.set()
cols = ['winPlacePerc', 'walkDistance', 'boosts', 'weaponsAcquired', 'damageDealt', 'killPlace']
sns.pairplot(train[cols], size = 2.5)
plt.show()

- 이상 배틀그라운드의 몇몇 피쳐들에 대한 시각화를 진행해 보았다.
- 위의 시각화를 기반으로 Feature Engineering을 진행한다면 유의미한 결과를 도출해 낼 수 있을 것으로 기대한다.